# WORKFLOW

## Step 1: Importing Libraries & The Original Data

---

## Step 2: Exploratory Data Analysis: Understanding the Original Data

**We will analyse and plot:**

2.1.   the number of cars per brand to get a feeling of the distirbution of the dataset under the *'brand'* column

2.2.   the distribution of the model year

2.3.   the distribution of the milage

2.4.   the distribution of prices

2.5.   the various fuel types, engines, and transmissions

2.6.   the vehicles with a reported accident vs those that have not reported one

---

## Step 3: Data Cleaning, Pre-Processing & Feature Engineering

3.1   removal of redundant features following the EDA

3.2   dropping records with limited statistical importance while ensuring this does not result in a certain class being underrepresented, subsequently leading to an imbalanced dataset

3.3. Missing data under the Engine Volume, HP, and/or Cylinders columns and the car is electric, the former and the latter will be set to 0.

3.4. We drop the 'clean_title' column since it contains a single unique value across all records and hence adds no value

3.5. We drop the interior and exterior colour columns as they are believed to not add any statistical significance and are irrelevant given the project scope

3.6. categorical brackets for some numeric data to decrease dimensionality

3.7. We create new columns "Engine Volume", "HP", "Cylinders" by using the available data under the 'engine' column where that is applicable

---

## Step 4: Exploring the new features and cleaning if necessary

In [ ]:
import pandas as pd
import numpy as np

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Data Pre-Processing
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Model Training
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Model Evaluation
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)

sns.set(style = 'whitegrid')
sns.set_context('notebook')

data = pd.read_csv(r'C:\Users\VladislavManolo\OneDrive - MIP\Documenten\GitHub\Kaggle-Used-Cars-Price-Prediction\data\train.csv')

data.head()

# Step 2: Exploratory Data Analysis

## 2.1. Brand

In [ ]:
# the 20 brands with the most presence in the dataset
print("Number of cars: ", data.shape[0])
print("Number of brands: ", data['brand'].nunique())

sns.countplot(data = data, y = 'brand', order = data['brand'].value_counts().iloc[:20].index)
plt.title('Top 20: Most Cars per Brand')
plt.xlabel('Number of Cars')
plt.ylabel('Brand')
plt.show()

print('\n')

# the 20 brands with the least presence in the dataset
sns.countplot(data = data, y = 'brand', order = data['brand'].value_counts().iloc[-20:].index)
plt.title('Top 20: Fewest Cars per Brand')
plt.xlabel('Number of Cars')
plt.ylabel('Brand')
plt.show()

## 2.2. Model Year

In [ ]:
# a histogram of the distirbution of the model year
sns.histplot(data['model_year'], kde = True, bins = 50)
plt.axvline(data['model_year'].mean(), color='red', linestyle='--', label = 'Average Year')
plt.title('Distribution Plot for Model Year')
plt.xlabel('Model Year')
plt.ylabel('Frequency')
plt.legend()
plt.show()

## 2.3. Milage

In [ ]:
# average mileage and price for each brand
brand_avg = data.groupby('brand', as_index = False).agg({'milage': 'mean', 'price': 'mean'})

# interactive plot
fig = px.scatter(brand_avg,
                 text='brand',
                 x='milage',
                 y='price',
                 title='Average Milage vs. Price by Brand',
                 labels={'mileage': 'Average Milage', 'price': 'Average Price'})

# marker size and label position
fig.update_traces(marker = dict(size = 4), textposition = 'top right', textfont_size = 9)
fig.show()

## 2.4. Price

In [ ]:
# a histogram of the distirbution of the price
sns.histplot(data['price'], kde = True, bins = 50)
plt.axvline(data['price'].mean(), color='red', linestyle='--', label = 'Average Price')
plt.title('Distribution Plot for Model Year')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.legend()
plt.show()

## 2.5. Price vs Model Year

In [ ]:
sns.lineplot(data = data, x = 'model_year', y = 'price')
plt.title('Price vs. Model Year')
plt.xlabel('Model Year')
plt.ylabel('Price')
plt.show()

## 2.6. Fuel

In [ ]:
# understanding the number of fuels across all vehicles
print(data.fuel_type.value_counts())

data[data['fuel_type'] ==  '–']['engine'].value_counts()

**Intermediate conclusion:**

*Missing data under the engine column, not necessarily as a NaN but as a '–' results in such under the 'fuel_type' column as well.*

## 2.7. Engine

In [ ]:
print("NaN values under the Engine column: ", data.engine.isna().sum())

data.engine.value_counts().sort_values(ascending = False)

electric_cars = data[data['engine'].str.contains('electric', case=False, na=False)]['engine'].count()
print(electric_cars)

# electric engines
electric_engines_counts = data[data['engine'].str.contains('electric', case=False, na=False)]['engine'].value_counts(ascending=False)
electric_engines_counts

In [ ]:
# print unique engine names with less than 10 characters
unique_engines_under_x_chars = [engine for engine in data['engine'].dropna().unique() if len(engine) < 20]

engine_counts_under_x_chars = data[data['engine'].isin(unique_engines_under_x_chars)]['engine'].value_counts()
engine_counts_under_x_chars

Many engines have 0HP declared under the 'engine' column. For electric vehicles HP shall be replaced with kW if such is applicable. 

For non-electric vehicles, it needs to be handled accordingly.

## 2.8. Transmission

In [ ]:
# set Manual to M/T
data['transmission'] = data['transmission'].replace('Manual', 'M/T')

# understanding the number of transmissions across all vehicles
print(data.transmission.value_counts().sort_values(ascending = False))

# Step 3: Data Cleaning, Pre-Processing & Feature Engineering

## 3.1. Addressing *fuel_type* data quality issues

In [ ]:
# fill under the 'fuel_type' column with the value 'Gasoline' if the word 'Gasoline' is mentioned under the 'engine' column
data.loc[data['engine'].str.contains('Gasoline', case = False), 'fuel_type'] = 'Gasoline'

# fill under the 'fuel_type' column with the value 'Hydrogen' if the word 'Hydrogen' is mentioned under the 'engine' column
data.loc[data['engine'].str.contains('Hydrogen', case = False), 'fuel_type'] = 'Hydrogen'

# drop records under the engine column where the value is '–'
data = data[data['engine'] != '–']

data.fuel_type.value_counts()

Addressing the last 2 remaining records labeled as '–' under the 'fuel_type' column

In [ ]:
data[data['fuel_type'] == '–']

In [ ]:
# make the two remaining records with '–' udner the 'fuel_type' column be 'Gasoline'
data.loc[data['fuel_type'] == '–', 'fuel_type'] = 'Gasoline'
data[data['fuel_type'] == '–']

Addressing the last remaining record labeled as 'not supported' under the 'fuel_type' column

In [ ]:
data[data['fuel_type'] == 'not supported']

In [ ]:
data.loc[data['fuel_type'] == 'not supported', 'fuel_type'] = 'Gasoline'
data[data['fuel_type'] == 'not supported']

In [ ]:
# Assuming df is your DataFrame and 'engine' is the column with engine descriptions
data.loc[data['engine'].str.contains('electric', case=False, na=False), 'fuel_type'] = 'Electric'
data.dropna(subset=['fuel_type'], inplace=True)

print("Remaining NaN columns under the 'fuel_type' column: ", data.fuel_type.isna().sum())
data.fuel_type.value_counts()

## 3.2. Dropping redundant columns

In [ ]:
# dropping redundant columns
data.drop(columns = ['clean_title', 'int_col', 'ext_col'], inplace = True)

# addressing inconsistensies among Tesla vehicles
data.loc[(data['brand'] == 'Tesla') & ( (data['fuel_type'] == 'Gasoline') | (data['fuel_type'] == 'Diesel') ), 'fuel_type'] = 'Electric'

# retrieving the HP, engine volume, and cylinders of each vehicle if such information exists under the 'engine' column
data['HP'] = data['engine'].str.extract('(\d+)\.?\d*HP') #fillna('NA')
data['Engine Volume (Liters)'] = data['engine'].str.extract('(\d+\.?\d*)L')
data['Cylinders'] = data['engine'].str.extract('(\d+)\s*Cylinder|V(\d+)').bfill(axis=1).iloc[:, 0]
data.drop(columns = ['engine'], inplace = True)

print(f"Dataframe shape: {data.shape}")

print(data.isna().sum(), '\n')

print(data.dtypes, '\n')

### 3.3. Milage & Price

In [ ]:
####################################################### milage & price categorical labels #######################################################
milage_brackets = [100, 25000, 58000, 95000, float('inf')]
milage_labels = ['Low Milage (100-25k)', 'Low-to-Medium Milage (25k-58k)', 'Medium-to-High Milage (58k-95k)', 'High Milage (95k+)']

price_brackets = [2000, 17000, 31000, 50000, float('inf')]
price_labels = ['Low End (2k-17k)', 'Low-to-Medium End (17k-31k)', 'Medium-to-High End (31k-50k)', 'High End (50k+)']

# Apply pd.cut with meaningful labels
data['milage_bracket'] = pd.cut(data['milage'], bins=milage_brackets, labels=milage_labels, include_lowest=True)
data['price_bracket'] = pd.cut(data['price'], bins=price_brackets, labels=price_labels, include_lowest=True)

data.head()

## 3.4. Analysing Accidents

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='accident', data=data)
plt.title('Accident Distribution')
plt.ylabel('Count')
plt.show()

data['accident'].replace({'None reported': 0, 'At least 1 accident or damage reported': 1}, inplace = True)
print("NaN records under the 'accident' column: ", len(data[data['accident'].isna()]))

data.dropna(subset = ['accident'], inplace = True)

sns.heatmap(data[num_features].corr(), annot = True)

num_features = ['model_year', 'milage', 'price', 'accident']
data[num_features].describe().apply(lambda s: s.apply('{0:.0f}'.format))

## 3.5. Model Year Brackets

In [ ]:
current_year = 2024

# Calculate the car age
data['car_age'] = current_year - data['model_year']

# Classify into 4 categories
data['age_category'] = pd.cut(data['car_age'], bins=[-1, 5, 10, 20, float('inf')], labels=['0-5 years', '6-10 years', '11-20 years', '20+ years'])

data.head()

In [ ]:
data.to_csv('cleaned_train.csv')

# Data Pre-Processing & Feature Engineering

1. Handling Outliers

2. Feature Scaling of numeric features & one-hot encoding of categorical features

3. Cross Validation

4. Hyperaparmeter Tuning through GridSearchCV or RandomSearchCV

5. Fit-transform on train data and only transform on test data


# Model Training

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

# Define numeric and categorical columns
numeric_cols = ['model_year', 'milage', 'accident', 'car_age']
categorical_cols = ['brand', 'model', 'fuel_type', 'transmission']

# Split the data into train and test sets
X = data[numeric_cols + categorical_cols]
y = data['price']  # Assuming 'price' is the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing for numeric data
numeric_transformer = StandardScaler()

# Preprocessing for categorical data
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Custom class to add tqdm progress bar support for RandomForest
class RandomForestWithProgress(RandomForestRegressor):
    def fit(self, X, y, sample_weight=None):
        # Initialize tqdm progress bar
        self.n_estimators_ = 0  # Add this attribute for tracking progress
        for _ in tqdm(range(self.n_estimators), desc="Fitting Random Forest", leave=True):
            super().fit(X, y, sample_weight=sample_weight)
            self.n_estimators_ += 1  # Update the estimator count
        return self

# Create the pipeline with RandomForestRegressor
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestWithProgress(n_estimators=100, random_state=42, warm_start=True))
])

# Train the model with progress bar
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")

# --- Additional Evaluations and Graphs ---

# Feature importance plot
feature_names = numeric_cols + list(model.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(categorical_cols))
feature_importances = model.named_steps['regressor'].feature_importances_

plt.figure(figsize=(10, 6))
plt.barh(feature_names, feature_importances)
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.title("Feature Importance in Random Forest")
plt.show()

# Residual analysis
residuals = y_test - y_pred

plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.show()

# Distribution of residuals
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=30, edgecolor='k', alpha=0.7)
plt.xlabel("Residual")
plt.ylabel("Frequency")
plt.title("Distribution of Residuals")
plt.show()
